In [1]:
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import itertools as it
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression as mir
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.metrics import mean_absolute_percentage_error as mape
from statsmodels.tsa.stattools import pacf
import xgboost as xgb
from itertools import product
from scipy import signal
from scipy import stats
from statsmodels.tsa.deterministic import Fourier
from sklearn.model_selection import TimeSeriesSplit

In [2]:
de_train_final = pd.read_csv('./data/de_train_final.csv')
y_de_train = pd.read_csv('./data/y_de_train.csv')
fr_train_final = pd.read_csv('./data/fr_train_final.csv')
y_fr_train = pd.read_csv('./data/y_fr_train.csv')

de_test_final = pd.read_csv('./data/de_test_final.csv')
fr_test_final = pd.read_csv('./data/fr_test_final.csv')

In [3]:

print("de_train_final with shape:", de_train_final.shape)
print("y_de_train with shape:", y_de_train.shape)
print("fr_train_final with shape:", fr_train_final.shape)
print("y_fr_train with shape:", y_fr_train.shape)
print("de_test_final with shape:", de_test_final.shape)
print("fr_test_final with shape:", fr_test_final.shape)

de_train_final with shape: (643, 35)
y_de_train with shape: (643, 1)
fr_train_final with shape: (851, 35)
y_fr_train with shape: (851, 1)
de_test_final with shape: (289, 35)
fr_test_final with shape: (365, 35)


In [4]:
# Assuming 'TARGET' is your target variable
X = de_train_final
y = y_de_train

# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

# Train the model
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(x_test)

# Calculate Spearman Correlation
spearman_corr_lr, _ = spearmanr(y_test, y_pred_lr)
spearman_corr_lr_value = spearman_corr_lr.item()

# Print
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr_value:.1%}")

Spearman Correlation for Linear Regression: 71.0%


In [5]:
# Split the data

X_de = de_train_final
y_de = y_de_train

# Split the data
X_trainde, X_testde, Y_trainde, Y_testde = train_test_split(X_de, y_de, test_size=0.30, random_state=42)

X_fr = fr_train_final
y_fr = y_fr_train

# Split the data
X_trainfr, X_testfr, Y_trainfr, Y_testfr = train_test_split(X_fr, y_fr, test_size=0.30, random_state=42)

In [6]:
from scipy.stats import spearmanr
import numpy as np

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def metric_train(output, truth):
    return spearmanr(output, truth).correlation


def get_model(model_name, best_param=None):
    if model_name == 'dt':
        model = DecisionTreeRegressor(**(best_param if best_param else {}))
    elif model_name == 'bagging_ridge':
        base_model = Ridge(**(best_param['base_model'] if best_param and 'base_model' in best_param else {}))
        model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, **(best_param['model'] if best_param and 'model' in best_param else {}))
    elif model_name == 'extra_trees':
        model = ExtraTreesRegressor(**(best_param if best_param else {}))
    elif model_name == 'rf':
        model = RandomForestRegressor(**(best_param if best_param else {}))
    elif model_name == 'bagging_knn':
        base_model = KNeighborsRegressor(**(best_param['base_model'] if best_param and 'base_model' in best_param else {}))
        model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, **(best_param['model'] if best_param and 'model' in best_param else {}))
    elif model_name == 'bagging_svr':
        base_model = SVR(**(best_param['base_model'] if best_param and 'base_model' in best_param else {}))
        model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, **(best_param['model'] if best_param and 'model' in best_param else {}))
    elif model_name == 'bagging_linear':
        base_model = LinearRegression(**(best_param['base_model'] if best_param and 'base_model' in best_param else {}))
        model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, **(best_param['model'] if best_param and 'model' in best_param else {}))
    elif model_name == 'adaboost':
        model = AdaBoostRegressor(**(best_param if best_param else {}))
    elif model_name == 'gradient_boosting':
        model = GradientBoostingRegressor(**(best_param if best_param else {}))
    elif model_name == 'xgboost':
        model = XGBRegressor(**(best_param if best_param else {}))
    else:
        raise ValueError('Unknown Model')
    return model
        
scorer_train = make_scorer(metric_train)

In [ ]:
model_names = [
    'dt',  # Decision Tree Regressor
    'bagging_ridge',  # Bagging model based on Ridge regression
    'extra_trees',  # Extra Trees Regressor
    'rf',  # Random Forest Regressor
    'bagging_knn',  # Bagging model based on KNN regression
    'bagging_svr',  # Bagging model based on SVR
    'bagging_linear',  # Bagging model based on Linear regression
    'adaboost',  # AdaBoost Regressor
    'gradient_boosting' , # Gradient Boosting Regressor
    'xgboost' #Xgboost Regressor
]

# Train and evaluate models
results = []

for model_name in model_names:
    model = get_model(model_name)
    
    # Train on DE dataset
    model.fit(X_trainde, Y_trainde)
    predictions_de = model.predict(X_testde)
    score_de = metric_train(predictions_de, Y_testde)
    
    # Train on FR dataset
    model.fit(X_trainfr, Y_trainfr)
    predictions_fr = model.predict(X_testfr)  
    score_fr = metric_train(predictions_fr, Y_testfr)
    
    # Overall Score
    predictions_overall = np.concatenate((predictions_de, predictions_fr))
    truth_overall = np.concatenate((Y_testde, Y_testfr))
    score_overall = metric_train(predictions_overall, truth_overall)
    
    results.append({
        'Model': model_name,
        'DE_Score': score_de,
        'FR_Score': score_fr,
        'Overall_Score': score_overall  
    })

# Convert results to DataFrame for better visualization
import pandas as pd
results_df = pd.DataFrame(results)

In [8]:
print(results_df)

               Model  DE_Train_Score  FR_Train_Score  Overall_Score
0                 dt        0.494307        0.074232       0.270040
1      bagging_ridge        0.768485        0.232256       0.526632
2        extra_trees        0.705980        0.085086       0.397630
3                 rf        0.654511        0.122941       0.390540
4        bagging_knn       -0.242485        0.123651      -0.042472
5        bagging_svr       -0.075682        0.089942      -0.035366
6     bagging_linear        0.772510        0.228310       0.525331
7           adaboost        0.616103        0.136967       0.342104
8  gradient_boosting        0.661099        0.059797       0.367391
9            xgboost        0.698499        0.084659       0.398867


## Tuning

In [ ]:
from mlxtend.regressor import StackingCVRegressor

from sklearn.linear_model import RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

best_models = results_df.nlargest(5, 'Overall_Score')['Model']

estimators = []
for model_name in best_models:
    if model_name == 'dt':
        estimator = DecisionTreeRegressor()
    elif model_name == 'bagging_ridge':
        estimator = BaggingRegressor(base_estimator=RidgeCV())
    elif model_name == 'extra_trees':
        estimator = ExtraTreesRegressor()
    elif model_name == 'rf':
        estimator = RandomForestRegressor()
    elif model_name == 'bagging_knn':
        estimator = BaggingRegressor(base_estimator=KNeighborsRegressor())
    elif model_name == 'bagging_svr':
        estimator = BaggingRegressor(base_estimator=SVR())
    elif model_name == 'bagging_linear':
        estimator = BaggingRegressor(base_estimator=LinearRegression())
    elif model_name == 'adaboost':
        estimator = AdaBoostRegressor()
    elif model_name == 'gradient_boosting':
        estimator = GradientBoostingRegressor()
    elif model_name == 'xgboost':
        estimator = XGBRegressor()
    else:
        raise ValueError('Unknown Model')
    
    estimators.append((model_name, estimator))

# Define Stacking Model
stacking_model = StackingCVRegressor(regressors=[estimator for _, estimator in estimators], 
                                     meta_regressor=RidgeCV(),
                                     cv=5,
                                     use_features_in_secondary=True,
                                     random_state=42)

# Train and evaluate models
results = []

model_name = 'stacking_model'

model = stacking_model
    
# Train on DE dataset
model.fit(X_trainde, Y_trainde)
predictions_de = model.predict(X_testde)
score_de = metric_train(predictions_de, Y_testde)

# Train on FR dataset
model.fit(X_trainfr, Y_trainfr)
predictions_fr = model.predict(X_testfr)  
score_fr = metric_train(predictions_fr, Y_testfr)

# Overall Score
predictions_overall = np.concatenate((predictions_de, predictions_fr))
truth_overall = np.concatenate((Y_testde, Y_testfr))
score_overall = metric_train(predictions_overall, truth_overall)

results.append({
    'Model': model_name,
    'DE_Score': score_de,
    'FR_Score': score_fr,
    'Overall_Score': score_overall  
})

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)


In [15]:
print(results_df)

            Model  DE_Train_Score  FR_Train_Score  Overall_Score
0  stacking_model        0.719765        0.255731       0.498399


In [21]:
de_train_pred = stacking_model.predict(X_de)
de_train = de_train_final.reset_index()
de_train['TARGET'] = de_train_pred
de_train = de_train[['ID', 'TARGET']]


/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [20]:
de_test = stacking_model.predict(de_test_final)
de_test_pred = de_test_final.reset_index()
de_test_pred['TARGET'] = de_test
de_test_pred = de_test_pred[['ID', 'TARGET']]

/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [22]:
fr_train_pred = stacking_model.predict(X_fr)
fr_result = fr_train_final.reset_index()
fr_result['TARGET'] = fr_train_pred
fr_result = fr_result[['ID', 'TARGET']]

/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [23]:
fr_test = stacking_model.predict(fr_test_final)
fr_test_pred = fr_test_final.reset_index()
fr_test_pred['TARGET'] = fr_test
fr_test_pred = fr_test_pred[['ID', 'TARGET']]

/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [24]:
df_test = pd.read_csv('../challenge_data/X_test.csv')
test_pred = pd.DataFrame()
test_pred['ID'] = df_test['ID']

test_pred = test_pred.merge(de_test_pred[['ID', 'TARGET']], on='ID', how='left')
test_pred = test_pred.merge(fr_test_pred[['ID', 'TARGET']], on='ID', how='left')
test_pred['TARGET'] = test_pred['TARGET_x'].combine_first(test_pred['TARGET_y'])
test_pred = test_pred.drop(['TARGET_x', 'TARGET_y'], axis=1)

In [25]:
test_pred

,ID,TARGET
0,1115,-0.078927
1,1202,-0.347796
2,1194,0.512974
3,1084,0.095376
4,1135,-0.073170
...,...,...
649,879,-2.097163
650,673,-0.910960
651,1641,0.215476
652,712,4.582160


In [ ]:
test_pred.to_csv('./Submission/' + 'stack1_test' + '.csv', index=False)